<a href="https://colab.research.google.com/github/MUKESH-SK/Machine-Learning-Deep-Learning-Projects/blob/main/Gym_Tracker_%2CKneeBend_Count_using_mediapipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 15.8 MB/s eta 0:00:00


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
# import cv2
# import time
# import mediapipe as mp

# Build a calculator graph for MediaPipe
calculator_graph = mp.CalculatorGraph()

# Add the MediaPipe pose model calculator to the graph
pose_model = calculator_graph.add_calculator(
    "pose_model", "PoseModelCalculator")

# Add the custom calculator for counting reps and holding timer
rep_counter = calculator_graph.add_calculator(
    "rep_counter", "RepCounterCalculator")

# Connect the pose model output to the rep counter input
calculator_graph.connect(pose_model.outputs[0], rep_counter.inputs[0])

# Start the MediaPipe calculator graph
calculator_graph.start_watches()

# Open the video file
video = cv2.VideoCapture("path/to/video.mp4")

# Read frames from the video file
while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break
    calculator_graph.add_packet_to_input_stream(
        "video_frames", mp.packets.Image(frame))
    pose_output = calculator_graph.get_output("pose_model").get()
    knee_angle = calculate_knee_angle(pose_output) # Custom function to calculate knee angle
    if knee_angle < 140: # Check if knee is bent
        start_time = time.time()
        while (time.time() - start_time) < 8: # Hold timer limit of 8 seconds
            pose_output = calculator_graph.get_output("pose_model").get()
            knee_angle = calculate_knee_angle(pose_output)
            if knee_angle > 140: # Check if knee is stretched
                rep_count += 1
                break
            if (time.time() - start_time) >= 8: # Check if holding time limit is exceeded
                print("Keep your knee bent") # Feedback message
    frames.append(frame) # Add the frame to the recorded video

# Get the output from the rep counter calculator
rep_count = calculator_graph.get_output("rep_counter").get()

# Print the successful rep count
print("Successful rep count: ", rep_count)

# Stop the MediaPipe calculator graph
calculator_graph.stop_watches()

# Save the recorded video
out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*'MJPG'), 15, (frame.shape[1], frame.shape[0]))
for f in frames:
    out.write(f)
out.release()



ValueError: ignored